In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:80% !important; }</style>'))

In [2]:
import pandas as pd
import numpy as np

In [3]:
abilities = 'csv_files/db_22_3_excel/Abilities.xlsx'
activities = 'csv_files/db_22_3_excel/Work Activities.xlsx'
knowledge = 'csv_files/db_22_3_excel/Knowledge.xlsx'
skills = 'csv_files/db_22_3_excel/Skills.xlsx'

In [4]:
def process_df(filepath, new_column):
    from string import punctuation
    temp = pd.read_excel(filepath)
    t = [''.join([x for x in column if x not in punctuation]) 
             for column in temp.columns]
    temp.columns = [c.lower().replace(' ', '_') for c in t]
    elements = sorted(temp['element_name'].unique())
    temp['element_code'] = temp['element_name'].replace(to_replace=elements,
                                                        value=np.arange(1, 
                                                            len(elements)+1))
    temp = temp.loc[temp['scale_id'] == 'IM', :]
    temp.drop(['element_id', 'element_name', 'scale_id', 'scale_name', 'n',
               'standard_error', 'lower_ci_bound', 'upper_ci_bound',
               'recommend_suppress', 'not_relevant', 'date', 'domain_source'],
              axis=1, inplace=True)
    temp = temp.pivot(index=['onetsoc_code', 'title'],
                      columns=['element_code'],
                      values='data_value')
    temp.columns = [new_column+'_'+str(i) for i in range(1, temp.shape[1]+1)]
    temp.reset_index(level=1, inplace=True)
    temp.index.name = 'onetsoccode'
    temp.columns.name = None
    return temp

In [5]:
df_abilities = process_df(abilities, 'ability')
df_activities = process_df(activities, 'activity')
df_knowledge = process_df(knowledge, 'knowledge')
df_skills = process_df(skills, 'skill')

In [6]:
df_onet = df_abilities.merge(df_activities, on='title',
                             left_index=True, right_index=True)
df_onet = df_onet.merge(df_knowledge, on='title',
                        left_index=True, right_index=True)
df_onet = df_onet.merge(df_skills, on='title',
                        left_index=True, right_index=True)

In [7]:
df_onet.to_csv('onet_numeric.csv')